## stacking 集成算法：最佳模型
基础学习器抽取特征，次级学习器融合。5折。
- 4 ExtraTreesRegressor
- 5 Gradient Boosting Regressor
- 6 RandomForestRegressor
- 7 BaggingRegressor

In [1]:
# 导入必要的工具包
import numpy as np #用于数值计算
import pandas as pd #用于数据表处理，数据文件读写
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from sklearn import ensemble

import matplotlib.pyplot as plt #用于绘图：分析结果的可视化。
#应该是设置浮点数的形式格式，小数点后三位
pd.set_option('display.float_format', lambda x: '%.4f' % x)
#这一行是干嘛的？ 设置 matplotlib 让绘制的图形出现在 Notebook 里而不是新窗口
%matplotlib inline

In [2]:
def get_train_val(val_id):
    '''获取训练集和校验集 
    val_id = 0,1,2 #当前校验集编号 '''
    print('val_id=%s'% (val_id))
    df_val = train_list[val_id]
    train_tmp = []

    for i in range(1, train_part_num):
        cur_id = (val_id + i) % 5
        print('cur_id=%s'% (cur_id))
        train_tmp.append(train_list[cur_id])
    
#     只生成一次，节省内存 
    df_train = pd.concat(train_tmp)
    print('df_train.shape=%s  df_val.shape=%s'% (df_train.shape, df_val.shape))
    return df_train, df_val

In [3]:
Y_Standard = True
path_train = '../data/new/train=all/'
path_test = '../data/new/test/'

val_path = '../data/val/' #保存校验集测试结果
val_name = '%sval=stacking-%s-%s.csv'
summit_path = '../data/summit/stacking2/' #保存提交文件，测试集测试结果 
out_name = '%ssummit=stacking-%s-%s.csv' # 生成的结果文件名称
base = 'base2' #学习器级别 

train_part_num = 5 #训练集拆分个数
target = 'prediction_pay_price'
user_id = 'user_id'

## 指定读取数据类型，节省内存

In [4]:
col_int32 = ['user_id', 'register_time', 'wood_add_value', 'wood_reduce_value',
       'stone_add_value', 'stone_reduce_value', 'ivory_add_value',
       'ivory_reduce_value', 'meat_add_value', 'meat_reduce_value',
       'magic_add_value', 'magic_reduce_value', 'infantry_add_value',
       'infantry_reduce_value', 'cavalry_add_value', 'cavalry_reduce_value',
       'shaman_add_value', 'shaman_reduce_value', 'wound_infantry_add_value',
       'wound_infantry_reduce_value', 'wound_cavalry_add_value',
       'wound_cavalry_reduce_value', 'wound_shaman_add_value',
       'wound_shaman_reduce_value', 'general_acceleration_add_value',
       'general_acceleration_reduce_value', 'building_acceleration_add_value',
       'building_acceleration_reduce_value', 'reaserch_acceleration_add_value',
       'reaserch_acceleration_reduce_value', 'training_acceleration_add_value',
       'training_acceleration_reduce_value', 'treatment_acceleraion_add_value',
       'treatment_acceleration_reduce_value', 'bd_training_hut_level',
       'bd_healing_lodge_level', 'bd_stronghold_level',
       'bd_outpost_portal_level', 'bd_barrack_level',
       'bd_healing_spring_level', 'bd_dolmen_level', 'bd_guest_cavern_level',
       'bd_warehouse_level', 'bd_watchtower_level', 'bd_magic_coin_tree_level',
       'bd_hall_of_war_level', 'bd_market_level', 'bd_hero_gacha_level',
       'bd_hero_strengthen_level', 'bd_hero_pve_level', 'sr_scout_level', 
       'sr_training_speed_level', 'sr_infantry_tier_2_level',
       'sr_cavalry_tier_2_level', 'sr_shaman_tier_2_level',
       'sr_infantry_atk_level', 'sr_cavalry_atk_level', 'sr_shaman_atk_level',
       'sr_infantry_tier_3_level', 'sr_cavalry_tier_3_level',
       'sr_shaman_tier_3_level', 'sr_troop_defense_level',
       'sr_infantry_def_level', 'sr_cavalry_def_level', 'sr_shaman_def_level',
       'sr_infantry_hp_level', 'sr_cavalry_hp_level', 'sr_shaman_hp_level',
       'sr_infantry_tier_4_level', 'sr_cavalry_tier_4_level',
       'sr_shaman_tier_4_level', 'sr_troop_attack_level',
       'sr_construction_speed_level', 'sr_hide_storage_level',
       'sr_troop_consumption_level', 'sr_rss_a_prod_levell',
       'sr_rss_b_prod_level', 'sr_rss_c_prod_level', 'sr_rss_d_prod_level',
       'sr_rss_a_gather_level', 'sr_rss_b_gather_level',
       'sr_rss_c_gather_level', 'sr_rss_d_gather_level', 'sr_troop_load_level',
       'sr_rss_e_gather_level', 'sr_rss_e_prod_level',
       'sr_outpost_durability_level', 'sr_outpost_tier_2_level',
       'sr_healing_space_level', 'sr_gathering_hunter_buff_level',
       'sr_healing_speed_level', 'sr_outpost_tier_3_level',
       'sr_alliance_march_speed_level', 'sr_pvp_march_speed_level',
       'sr_gathering_march_speed_level', 'sr_outpost_tier_4_level',
       'sr_guest_troop_capacity_level', 'sr_march_size_level',
       'sr_rss_help_bonus_level', 'pvp_battle_count', 'pvp_lanch_count',
       'pvp_win_count', 'pve_battle_count', 'pve_lanch_count', 'pve_win_count',
       'pay_count']
col_float32 = ['avg_online_minutes', 'pay_price']

In [5]:
# 指定读取数据类型，节省内存
dtype_test = {}
for col in col_int32:
    dtype_test[col] = np.int32
for col in col_float32:
    dtype_test[col] = np.float32
# dtype_test

In [6]:
dtype_train = dtype_test.copy()
dtype_train[target] = np.float32
# dtype_train

## 读取数据，一次读取多次使用，不同组合 

In [7]:
train_list = []
for i in range(train_part_num):
    cur_id = i + 1
    train_list.append(pd.read_csv('%strain=fe-%s.csv' % (path_train, cur_id), dtype=dtype_train))
train_list[0].head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,332994,112,43413,18424,0,0,0,0,62775,9444,...,0,0,0,0,0,0,0.5000,0.0000,0,0.0000
1,2186887,136,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.3333,0.0000,0,0.0000
2,383991,134,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.6667,0.0000,0,0.0000
3,102315,134,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.5000,0.0000,0,0.0000
4,225310,106,28975,5780,0,0,0,0,33525,3220,...,0,0,0,0,0,0,0.6667,0.0000,0,0.0000


## 5折交叉验证

In [8]:
# 根据校验集编号 0，1,2,3,4 ，获取训练集和校验集
df_train, df_val = get_train_val(4)

# 从原始数据中分离输入特征x和输出y
train_y = df_train[target].values
train_X = df_train.drop([target, user_id], axis = 1)
val_y = df_val[target].values
val_X = df_val.drop([target, user_id], axis = 1)

val_id=4
cur_id=0
cur_id=1
cur_id=2
cur_id=3
df_train.shape=(1830405, 109)  df_val.shape=(457602, 109)


In [9]:
# 数据标准化：分别初始化对特征和目标值的标准化器 
ss_X = StandardScaler()
ss_y = StandardScaler()

# 分别对训练和测试数据的特征以及目标值进行标准化处理
train_X_ss = ss_X.fit_transform(train_X)
val_X_ss = ss_X.transform(val_X)

if Y_Standard:
    train_y_ss = ss_y.fit_transform(train_y.reshape(-1, 1))
    val_y_ss = ss_y.transform(val_y.reshape(-1, 1))

# 需要转换为一维数组 
train_y_ss_1d = train_y_ss.reshape(train_y_ss.shape[0])
val_y_ss_1d = val_y_ss.reshape(val_y_ss.shape[0])

In [10]:
def inverse_StandardScaler(predict):
    ''' 标准化数据还原  '''
    if Y_Standard:
        predict = ss_y.inverse_transform(predict)
        print('predict = ', predict)
    return predict

def generate_summit(predict):
    ''' 输出预测后的数据 '''
    testPredict = test.copy()
    testPredict[target] = predict
    testPredict = testPredict[[user_id, target]]
#     应该过滤掉负数
    testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

def generate_val_predict(predict):
    ''' 生成校验集预测后的数据 '''
    testPredict = df_val.copy()
    testPredict[target] = predict
    testPredict = testPredict[[user_id, target]]
#     应该过滤掉负数
    testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

## 模型训练
###  4. ExtraTreesRegressor

In [11]:
%%time
model = ensemble.ExtraTreesRegressor(n_estimators=16, min_samples_split=80, min_samples_leaf=8, n_jobs=-1)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 4min 6s

Wall time: 6min 47s


In [12]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss)
scoreVal = model.score(val_X_ss, val_y_ss)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss, predict_train) ** 0.5
predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss, predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=10, min_samples_split=80, min_samples_leaf=8 
# n_estimators=12, min_samples_split=80, min_samples_leaf=8
# n_estimators=14, min_samples_split=80, min_samples_leaf=8
# The score: Train=0.6521757828419238  Val=0.5987845196080509
# The RMSE: Train=0.5897663490208811  Val=0.43224438107351143
# n_estimators=16, min_samples_split=80, min_samples_leaf=8 

The score: Train=0.5756203255311486  Val=0.48576997881035494
The RMSE: Train=0.6514444283623049  Val=0.6309690932311239


### 保存校验集测试结果

In [13]:
print('predict_val= ', predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'ExtraTreesRegressor'), index=False)
valPredict.head()

predict_val=  [-0.01991728 -0.02015105 -0.02015979 ... -0.01995695 -0.02015841
 -0.02015806]
predict =  [0.02217546 0.00101233 0.00022061 ... 0.01858445 0.00034597 0.00037783]


,user_id,prediction_pay_price
0,1987367,0.0222
1,2292622,0.0010
2,466954,0.0002
3,1008130,0.0004
4,9759,0.0000


## 加载测试数据

In [14]:
test = pd.read_csv(path_test + 'tap_fun_test-fe.csv', dtype=dtype_test)
test_X = test.drop([user_id], axis = 1)
test_X_ss = ss_X.transform(test_X)
test.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,sr_rss_help_bonus_level,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count
0,14933,94,166415,138362,0,0,0,0,258522,90142,...,0,0,0,0,1,1,1,8.0000,0.0000,0
1,14934,94,10000,600,0,0,0,0,10000,400,...,0,0,0,0,0,0,0,0.1667,0.0000,0
2,14935,94,10000,3700,0,0,0,0,10000,2000,...,0,0,0,0,0,0,0,17.0000,0.0000,0
3,14936,94,210000,0,200000,0,200000,0,610000,0,...,0,0,0,0,0,0,0,1.6667,0.0000,0
4,14937,94,11500,3700,0,0,0,0,11000,2000,...,0,0,0,0,0,0,0,0.3333,0.0000,0


In [15]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)
# lasso_y_predict_test =  [0.4556051 0.4556051 0.4556051 ... 0.4556051 0.4556051 0.4556051]

model_y_predict_test =  [-0.02016139 -0.02016223 -0.02016223 ... -0.020055   -0.02016223
 -0.02016197]
predict =  [ 7.57668193e-05 -8.71155108e-08 -8.71155108e-08 ...  9.70718840e-03
 -8.71155108e-08  2.32751671e-05]


In [16]:
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'ExtraTreesRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,0.0001
1,14934,0.0000
2,14935,0.0000
3,14936,0.0000
4,14937,0.0000


## 5 GradientBoostingRegressor

In [17]:
%%time
model = ensemble.GradientBoostingRegressor(learning_rate=0.25, n_estimators=14, min_samples_split=80, min_samples_leaf=8, alpha=0.99)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 3min 2s 

Wall time: 2min 13s


In [18]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# learning_rate=0.1, n_estimators=50, min_samples_split=50, min_samples_leaf=10, alpha=0.8
# learning_rate=0.2, n_estimators=30, min_samples_split=50, min_samples_leaf=10, alpha=0.8
# learning_rate=0.2, n_estimators=20, min_samples_split=50, min_samples_leaf=10, alpha=0.8
# learning_rate=0.2, n_estimators=20, min_samples_split=50, min_samples_leaf=10, alpha=0.99
# learning_rate=0.25, n_estimators=16, min_samples_split=64, min_samples_leaf=8, alpha=0.99
# learning_rate=0.25, n_estimators=12, min_samples_split=64, min_samples_leaf=8, alpha=0.99
# learning_rate=0.25, n_estimators=12, min_samples_split=80, min_samples_leaf=8, alpha=0.99
# learning_rate=0.25, n_estimators=13, min_samples_split=80, min_samples_leaf=8, alpha=0.99
# learning_rate=0.25, n_estimators=14, min_samples_split=80, min_samples_leaf=8, alpha=0.99
# The score: Train=0.7474515268358191  Val=0.45818076230225474
# The RMSE: Train=0.5025420292903933  Val=0.8525285883738782

The score: Train=0.6811526105797193  Val=0.3464289280568956
The RMSE: Train=0.5646658447485722  Val=0.7113376300355244


### 保存校验集测试结果

In [19]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'GradientBoostingRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.01559045 -0.01559045 -0.01559045 ... -0.01559045 -0.01559045
 -0.01559045]
predict =  [0.41388658 0.41388658 0.41388658 ... 0.41388658 0.41388658 0.41388658]


,user_id,prediction_pay_price
0,1987367,0.4139
1,2292622,0.4139
2,466954,0.4139
3,1008130,0.4139
4,9759,0.4139


In [20]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.01559045 -0.01559045 -0.01559045 ... -0.01559045 -0.01559045
 -0.01559045]
predict =  [0.41388658 0.41388658 0.41388658 ... 0.41388658 0.41388658 0.41388658]


In [21]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'GradientBoostingRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,0.4139
1,14934,0.4139
2,14935,0.4139
3,14936,0.4139
4,14937,0.4139


## 6 RandomForestRegressor

In [22]:
%%time
model = ensemble.RandomForestRegressor(n_estimators=16, min_samples_split=80, min_samples_leaf=8, n_jobs=-1)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 10min 2s 

Wall time: 7min 54s


In [23]:
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=14, min_samples_split=80, min_samples_leaf=8, min_impurity_decrease=0.1
# n_estimators=20, min_samples_split=80, min_samples_leaf=8, min_impurity_decrease=0.1
# n_estimators=20, min_samples_split=80, min_samples_leaf=8, min_impurity_decrease=0.0
# n_estimators=20, min_samples_split=100, min_samples_leaf=8
# n_estimators=16, min_samples_split=100, min_samples_leaf=8
# The score: Train=0.6193906706679225  Val=0.4333588747424302
# The RMSE: Train=0.6169354557150862  Val=0.8718379844283086 
# n_estimators=16, min_samples_split=80, min_samples_leaf=8
# The score: Train=0.6393206199636329  Val=0.35457588882649244
# The RMSE: Train=0.6005657993266247  Val=0.517339714712821

The score: Train=0.60076477613784  Val=0.45951994837695753
The RMSE: Train=0.63185075978246  Val=0.6468732729373746


### 保存校验集测试结果

In [24]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'RandomForestRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.02012636 -0.0201608  -0.02015859 ... -0.02015874 -0.02016121
 -0.02016061]
predict =  [3.24703555e-03 1.29771838e-04 3.29886858e-04 ... 3.15924405e-04
 9.24195345e-05 1.46705193e-04]


,user_id,prediction_pay_price
0,1987367,0.0032
1,2292622,0.0001
2,466954,0.0003
3,1008130,0.0001
4,9759,0.0000


In [25]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.02016032 -0.02016063 -0.02016062 ... -0.02009732 -0.02016145
 -0.02016153]
predict =  [1.72601980e-04 1.44893029e-04 1.45679420e-04 ... 5.87600448e-03
 7.01564847e-05 6.30155483e-05]


In [26]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'RandomForestRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,0.0002
1,14934,0.0001
2,14935,0.0001
3,14936,0.0160
4,14937,0.0002


## 7 BaggingRegressor

In [27]:
%%time
model = ensemble.BaggingRegressor(n_jobs=2, n_estimators=20)
model.fit(train_X_ss, train_y_ss_1d)
# Wall time: 9min 2s 

Wall time: 17min 14s


In [28]:
%%time
# 模型评估
scoreTrain = model.score(train_X_ss, train_y_ss_1d)
scoreVal = model.score(val_X_ss, val_y_ss_1d)
print('The score: Train=%s  Val=%s'% (scoreTrain, scoreVal)) 

model_predict_train = model.predict(train_X_ss)
rmseTrain = mean_squared_error(train_y_ss_1d, model_predict_train) ** 0.5
model_predict_val = model.predict(val_X_ss)
rmseVal = mean_squared_error(val_y_ss_1d, model_predict_val) ** 0.5
print('The RMSE: Train=%s  Val=%s' % (rmseTrain, rmseVal))

# n_estimators=10 
# n_estimators=15 
# he score: Train=0.944126770370558  Val=0.35667035830687766
# The RMSE: Train=0.2363752321974746  Val=0.5473401373069637
# n_estimators=20 


The score: Train=0.913251997388679  Val=0.36646045612672906
The RMSE: Train=0.29453019728192303  Val=0.7003517799305795
Wall time: 1min 4s


### 保存校验集测试结果

In [29]:
print('model_predict_val= ', model_predict_val)
# 标准化数据还原 
predict_val = inverse_StandardScaler(model_predict_val)
valPredict = generate_val_predict(predict_val)
#输出预测后的数据
valPredict.to_csv(val_name %(val_path, base, 'BaggingRegressor'), index=False)
valPredict.head()

model_predict_val=  [-0.02016024 -0.02016089 -0.02015934 ... -0.02015791 -0.02016131
 -0.02016111]
predict =  [1.80489074e-04 1.20919390e-04 2.61997773e-04 ... 3.90591308e-04
 8.29102097e-05 1.01077875e-04]


,user_id,prediction_pay_price
0,1987367,0.0002
1,2292622,0.0001
2,466954,0.0003
3,1008130,0.0001
4,9759,0.0001


In [30]:
#对测试数据进行预测
model_y_predict_test = model.predict(test_X_ss)
print('model_y_predict_test = ', model_y_predict_test)

# 标准化数据还原 
y_predict_test = inverse_StandardScaler(model_y_predict_test)

model_y_predict_test =  [-0.02016037 -0.02016043 -0.02016078 ... -0.02016117 -0.02016132
 -0.02016098]
predict =  [1.68377059e-04 1.62723280e-04 1.31418879e-04 ... 9.59142042e-05
 8.26657229e-05 1.12932125e-04]


In [31]:
# ridge_y_predict_test = move_up(ridge_y_predict_test)
#输出预测后的数据
testPredict = generate_summit(y_predict_test)
#输出预测后的数据
testPredict.to_csv(out_name %(summit_path, base, 'BaggingRegressor'), index=False)
testPredict.head()

,user_id,prediction_pay_price
0,14933,0.0002
1,14934,0.0002
2,14935,0.0001
3,14936,0.0000
4,14937,0.0001


## 7-1 BaggingRegressor的代码最佳示例
https://www.helplib.com/Python-Module-Examples/article_66154